# Intersight Python SDK Installation

If you are familiar or proficient with Python, you can take advantage of the Intersight Python SDK available for install from the [Python Package Index](https://pypi.org/project/intersight/).

![Python SDK for Cisco Intersight](images/python_sdk_pip.png)

Notice the `pip install intersight` command. That's the command used to install the Python SDK in your environment.  Be sure you are using Python >= 3.6, earlier versions of Python are not supported.


In [ ]:
pip install intersight


> Be sure to uninstall any conflicting versions of the SDK if you have previous installs.  You can check installed versions with pip list.

In [ ]:
pip list

> If you see Intersight-OpenAPI installed, you can run `pip unistall Intersight-OpenAPI`

## API Keys and User Authentication

Now that the Intersight Python SDK is installed, let's use it to connect to our Intersight environment.  First, you'll need an Intersight API Key ID and secret (private) key from your Intersight account.  From the Settings menu in Intersight, Select API Keys and Generate API Key

![Generate API Key](images/generate_api_key.png)

A version 3 key can be used with the SDK and is recommended for long term compatibility with Intersight's API.

![Version 3 key](images/version_3_key.png)

The Generated API Key ID can be copied to the clipboard and used in API authentication below.  Also be sure to save the Secret (private) Key to a local file that only you have access to (a download option is provided by Intersight).

![Download Secret Key](images/secret_key_download.png)

### Using the Intersight Python SDK

To use the SDK, you must import required modules.  You can view help once you've imported any required modules.


In [ ]:
import intersight

# View the help
help(intersight)

### API Client Configuration

To use the SDK, you will need to configure an API client using the API Keys you created above.  You can view the help for Intesight's Configuration and ApiClient classes to see what is needed.

In [ ]:
help(intersight.Configuration)


In [ ]:
help(intersight.ApiClient)

### Example Usage

We're now ready to create an API client and use it to connect with Intersight.

> The values below only serve as an example so they won't work in your environment.  Be sure to enter the values for your account in the `key_id` variable below as well as the `private_key_path`.  Also be sure that you are using v3 API keys from your Intersight account.

In [ ]:
configuration = intersight.Configuration(
    signing_info=intersight.HttpSigningConfiguration(
        key_id='596cc79e5d91b400010d15ad/5db71f977564612d30cc3860/5e9217a57564612d302f475b',
        private_key_path='/Users/dsoper/Downloads/SecretKey.txt',
        signing_scheme=intersight.signing.SCHEME_HS2019,
        signed_headers=[intersight.signing.HEADER_HOST,
                        intersight.signing.HEADER_DATE,
                        intersight.signing.HEADER_DIGEST,
                        intersight.signing.HEADER_REQUEST_TARGET
                        ]
    )
)
api_client = intersight.ApiClient(configuration)


#### Querying the API

Now that you are authenticated, let's use the SDK to query the Alarms resource in Intersight.  The example below sets up a query filter to get any Critical alarms that have occurred in the last 7 days.

In [ ]:
from datetime import datetime, timedelta
import intersight.api.cond_api

# Get condition class instance
api_instance = intersight.api.cond_api.CondApi(api_client)

# Find all Critical severity alarms within the last 7 days
search_period = datetime.now() - timedelta(days=7)
# Convert date object to formatted date string for use with intersight queries
date_str = f"{search_period.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]}Z"
query_filter = f"Severity eq Critical and CreationTime gt {date_str}"
# Only include CreationTime, Description, Code, and Acknowledge in results
query_select = "CreationTime,Description,Code,Acknowledge"

# Get alarms using query parameters
alarm_query = api_instance.get_cond_alarm_list(filter=query_filter, select=query_select)

print(alarm_query)


### Updating Resources

Using data from the query above, we can update Alarm resources to acknowledge alarms.  To acknowledge an Alarm, set the acknowledge attribute to `Acknowledge`.

In [ ]:
import intersight.model.cond_alarm

alarm = intersight.model.cond_alarm.CondAlarm(acknowledge='Acknowledge')
alarm_patch = api_instance.patch_cond_alarm(moid=alarm_query.results[0].moid, cond_alarm=alarm)

You can re-run the query example to confirm that the patch operation has changed Acknowledge state on the alarm.

### Additional Examples

Several additional examples of using the SDK are on GitHub in the [Intersight Python Utilties repo](https://github.com/CiscoDevNet/intersight-python-utils).  The repo also contains a credentials.py module to simplify authentication across all of the example scripts.

### How to Learn More

Additional information on the Intersight API is available on the [Intersight API Docs Pages](https://www.intersight.com/apidocs).  Intersight also features an in system [API schema browser](https://www.intersight.com/apidocs/apirefs/) where developers can view the complete resource model and interact with the API in their account.  As an example, you can view the [schema for Alarms](https://www.intersight.com/apidocs/apirefs/cond/Alarms/model/) and if you are signed in to your Intersight account you can also perform operations against the API.

![API Reference Alarms](images/api_ref_alarms.png)

Your browser developer console can also be used to view API calls (all browser actions use the API).  In Chrome for example, you can right click and select Inspect to view the developer console.

![Chrome Inspect](images/chrome_inspect.png)

The Network tab shows details on all API calls performed by the browser including query strings used to retrieve specific data.

![Chrome Network Tab](images/chrome_network_tab.png)